In [1]:
import chess
import chess.pgn
import os

In [ ]:
def read_games_from_file(file_path):
    games = []
    with open(file_path, 'r') as pgn_file:
        while True:
            try:
                game = chess.pgn.read_game(pgn_file)
                if game is None:
                    break
                games.append(game)
            except ValueError as e:
                print(f"Pominięto partię z powodu błędu: {e}")
    return games

def is_checkmate(game):
    board = game.board()
    for move in game.mainline_moves():
        board.push(move)
    return board.is_checkmate()

def filter_checkmate_games(games, pgn_mate_path, pgn_draw_path):
    checkmate_games = []
    draw_games = []
    for cnt, game in enumerate(games):
        try:
            res = is_checkmate(game)
            if res:
                checkmate_games.append(game)
            elif game.headers["Result"] == "1/2-1/2":
                draw_games.append(game)
        except Exception as e:
            print(f"Pominięto partię numer {cnt} z powodu błędu: {e}")

    with open(pgn_mate_path, "a") as output_file:
        for game in checkmate_games:
            output_file.write(str(game))
            output_file.write("\n\n")

    with open(pgn_draw_path, "a") as output_file:
        for game in draw_games:
            output_file.write(str(game))
            output_file.write("\n\n")


data_path = "../data/raw/"

files = os.listdir(data_path)

# for file in files:
#     games = read_games_from_file(data_path + file)
#     filter_checkmate_games(games, data_path + "mates.pgn", data_path + "draw.pgn")

In [3]:
games = read_games_from_file(data_path+"mates.pgn")
drawn = read_games_from_file(data_path + "draw.pgn")

In [5]:


invalid = 0
for cnt, game in enumerate(drawn):
    data = []
    board = chess.Board()
    cor = True
    

    for move in game.mainline_moves():
        legal_moves = list(board.legal_moves)
        if move in legal_moves:
            board_fen = board.fen()
            board_fen += "5-5"+","
            board_fen += str(int(board.turn)) + "\n"
            data.append(board_fen)
            board.push(move)
        else:
            cor = False
            invalid += 1
            break
    if cor:
        with open(data_path+"drawn.txt", "a") as f:
            f.writelines(data)
    if cnt % 1000 == 0:
        print(f"{cnt}/{len(drawn)}")

        
print("invalid drawns: ", invalid)


0/82977
1000/82977
2000/82977
3000/82977
4000/82977
5000/82977
6000/82977
7000/82977
8000/82977
9000/82977
10000/82977
11000/82977
12000/82977
13000/82977
14000/82977
15000/82977
16000/82977
17000/82977
18000/82977
19000/82977
20000/82977
21000/82977
22000/82977
23000/82977
24000/82977
25000/82977
26000/82977
27000/82977
28000/82977
29000/82977
30000/82977
31000/82977
32000/82977
33000/82977
34000/82977
35000/82977
36000/82977
37000/82977
38000/82977
39000/82977
40000/82977
41000/82977
42000/82977
43000/82977
44000/82977
45000/82977
46000/82977
47000/82977
48000/82977
49000/82977
50000/82977
51000/82977
52000/82977
53000/82977
54000/82977
55000/82977
56000/82977
57000/82977
58000/82977
59000/82977
60000/82977
61000/82977
62000/82977
63000/82977
64000/82977
65000/82977
66000/82977
67000/82977
68000/82977
69000/82977
70000/82977
71000/82977
72000/82977
73000/82977
74000/82977
75000/82977
76000/82977
77000/82977
78000/82977
79000/82977
80000/82977
81000/82977
82000/82977
invalid drawns:  

KeyboardInterrupt: 

In [8]:
invalid = 0

for cnt, game in enumerate(games):
    data = []
    board = chess.Board()
    cor = True
    

    for move in game.mainline_moves():
        legal_moves = list(board.legal_moves)
        if move in legal_moves:
            board_fen = board.fen()
            res = game.headers['Result']
            board_fen += res + "," + str(int(board.turn)) + "\n"
            data.append(board_fen)

            opp_res = "1-0" if res == "0-1" else "0-1"
            opp_board = board.mirror()
            board_fen = opp_board.fen()
            board_fen += opp_res + "," + str(int(opp_board.turn)) + "\n"
            data.append(board_fen)

            board.push(move)
        else:
            cor = False
            invalid += 1
            break
    if cor:
        with open(data_path+"mates.txt", "a") as f:
            f.writelines(data)
    if cnt % 1000 == 0:
        print(f"{cnt}/{len(games)}")
print("invalid: ", invalid)

0/51366
1000/51366
2000/51366
3000/51366
4000/51366
5000/51366
6000/51366
7000/51366
8000/51366
9000/51366
10000/51366
11000/51366
12000/51366
13000/51366
14000/51366
15000/51366
16000/51366
17000/51366
18000/51366
19000/51366
20000/51366
21000/51366
22000/51366
23000/51366
24000/51366
25000/51366
26000/51366
27000/51366
28000/51366
29000/51366
30000/51366
31000/51366
32000/51366
33000/51366
34000/51366
35000/51366
36000/51366
37000/51366
38000/51366
39000/51366
40000/51366
41000/51366
42000/51366
43000/51366
44000/51366
45000/51366
46000/51366
47000/51366
48000/51366
49000/51366
50000/51366
51000/51366
invalid:  244


In [1]:
import torch
import chess
data_path = "../data/raw/"
def create_tensor(board: chess.Board, transform: bool=False):
    matrix_board = torch.zeros((6, 8, 8))
    for i in range(8):
        for j in range(8):
            piece = board.piece_at(chess.square(i, j))
            if piece is not None:
                piece_type = piece.piece_type
                piece_color = piece.color
                index = piece_type - 1

                row = 7-j if not transform else j

                if piece_color == chess.WHITE:
                    matrix_board[index, row, i] = 1
                else:
                    matrix_board[index, row, i] = -1
    if transform:
        matrix_board *= -1
        matrix_board = torch.where(torch.abs(matrix_board) < 1e-6, torch.zeros_like(matrix_board), matrix_board)
    return matrix_board


tensors = []
labels = []
with open(data_path+"mates.txt", "r") as f:
    data = f.readlines()

white_win = 0
black_win = 0
for idx, line in enumerate(data):
    fen, result, turn = line[:-6], line[-6:-3], line[-2]
    if result[0] == "1":
        white_win += 1
    else:
        black_win += 1
    tensor = create_tensor(chess.Board(fen))
    tensor = tensor.view(6*8*8)
    tensor = torch.cat([tensor, torch.tensor([int(turn)])])
    label = int(result[0])
    tensors.append(tensor)
    labels.append(label)

    if idx % 100000 == 0:
        print(f"{idx}/{len(data)}" )


0/15524326
100000/15524326
200000/15524326
300000/15524326
400000/15524326
500000/15524326
600000/15524326
700000/15524326
800000/15524326
900000/15524326
1000000/15524326
1100000/15524326
1200000/15524326
1300000/15524326
1400000/15524326
1500000/15524326
1600000/15524326
1700000/15524326
1800000/15524326
1900000/15524326
2000000/15524326
2100000/15524326
2200000/15524326
2300000/15524326
2400000/15524326
2500000/15524326
2600000/15524326
2700000/15524326
2800000/15524326
2900000/15524326
3000000/15524326
3100000/15524326
3200000/15524326
3300000/15524326
3400000/15524326
3500000/15524326
3600000/15524326
3700000/15524326
3800000/15524326
3900000/15524326
4000000/15524326
4100000/15524326
4200000/15524326
4300000/15524326
4400000/15524326
4500000/15524326
4600000/15524326
4700000/15524326
4800000/15524326
4900000/15524326
5000000/15524326
5100000/15524326
5200000/15524326
5300000/15524326
5400000/15524326
5500000/15524326
5600000/15524326
5700000/15524326
5800000/15524326
5900000/1552

In [2]:
print(f"black win: {black_win}")
print(f"white win: {white_win}")

black win: 7762163
white win: 7762163


In [3]:


draw_num = int((black_win + white_win) / 1)

with open(data_path+"drawn.txt", "r") as f:
    data = f.readlines()

import random

random.seed(42)
random.shuffle(data)

try:
    data = data[:draw_num]
except Exception:
    data = data

labels.extend([0.5 for _ in range(len(data))])

for line in data:
    fen, result, turn = line[:-6], line[-6:-3], line[-2]
    tensor = create_tensor(chess.Board(fen))
    tensor = tensor.view(6*8*8)
    tensor = torch.cat([tensor, torch.tensor([int(turn)])])
    tensors.append(tensor)


In [4]:

y = torch.tensor(labels)
torch.save(y, "../data/prep/y_turn.pt")

del y


In [5]:
x = torch.stack(tensors)
torch.save(x, "../data/prep/X_turn.pt")
del x

In [6]:
y = torch.tensor(labels)
count_0 = torch.sum(y == 0).item()
count_1 = torch.sum(y == 1).item()
count_05 = torch.sum(y == 0.5).item()

print(f"Liczba zer: {count_0}")
print(f"Liczba jedynek: {count_1}")
print(f"Liczba 0.5: {count_05}")

Liczba zer: 7762163
Liczba jedynek: 7762163
Liczba 0.5: 11928218
